In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from datetime import datetime
import mne
from mne.preprocessing import ICA
from scipy import signal
import warnings
import traceback

warnings.filterwarnings("ignore", category=RuntimeWarning)

log_path = "data_logs/Aws Safi"
data_path = "data"

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)


In [12]:
def mp(c):
    """Convert log color markers to valid Plotly colors"""
    if c.strip()=="'blue'":
        return 'blue'
    if c.strip()=="'lightblue'":
        return 'red'
    return 'black'

logs = {}
for log_file in os.listdir(log_path):
    if not log_file.endswith('.txt'):
        continue
    identifier = log_file.split("_run")[1].split(".")[0]
    try:
        with open(f"{log_path}/{log_file}", "r") as f:
            a = f.readlines()
            if not a:
                print(f"WARNING: Empty log file: {log_file}")
                continue
            logs[identifier] = [
                (datetime.strptime(line.split(" ")[0], "[%Y-%m-%dT%H:%M:%S.%fZ]").timestamp(),
                 mp(line.split(" ")[-1]),
                 " ".join(line.split(" ")[2:]))
                for line in a
                if 'blue' in line or '+' in line
            ]
    except Exception as e:
        print(f"ERROR parsing log file {log_file}: {str(e)}")

print(f"Successfully parsed {len(logs)} log files")

def validate_csv(file_path):
    """Validate if a CSV file is readable and has data"""
    try:
        if not os.path.exists(file_path):
            return False, "File does not exist"

        if os.path.getsize(file_path) == 0:
            return False, "File is empty"

        with open(file_path, 'r') as f:
            first_lines = []
            for i, line in enumerate(f):
                if i >= 5:
                    break
                first_lines.append(line)

            if not first_lines:
                return False, "File has no content"

            if not ',' in first_lines[0]:
                return False, "File does not appear to be a valid CSV"
            l = f.readlines()
            if not l[0].startswith('Timestamp'):
                l = l[1:]
            with open(f"{file_path}", 'w') as f:
                f.writelines(l)

        try:
            df_sample = pd.read_csv(file_path, nrows=5)
            if df_sample.empty:
                return False, "Pandas could not read any rows"
            return True, "Valid"
        except pd.errors.EmptyDataError:
            return False, "No columns to parse from file"
        except pd.errors.ParserError:
            return False, "Parser error - file may be corrupted"
        except Exception as e:
            return False, f"Error reading with pandas: {str(e)}"

    except Exception as e:
        return False, f"Validation error: {str(e)}"

Successfully parsed 3 log files


In [14]:
def preprocess_eeg(file_path, identifier, logs):
    """Process EEG data and print findings"""
    print(f"\n{'='*80}\nProcessing file: {os.path.basename(file_path)}\n{'='*80}")

    try:
        try:
            df = pd.read_csv(file_path)
            print(f"Successfully read data: {df.shape[0]} rows, {df.shape[1]} columns")
        except Exception as e:
            print(f"ERROR: Could not read data: {str(e)}")
            return None

        if df.empty:
            print("ERROR: DataFrame is empty")
            return None

        if 'Timestamp' in df.columns:
            df.set_index('Timestamp', inplace=True)
            print("Set 'Timestamp' as index")

        eeg_channels = [col for col in df.columns if col.startswith('EEG.') and
                       not any(x in col for x in ['Counter', 'Interpolated', 'RawCq', 'Battery', 'MarkerHardware'])]

        if not eeg_channels:
            print("ERROR: No EEG channels found in data")
            return None

        print(f"Found {len(eeg_channels)} EEG channels: {', '.join(eeg_channels)}")

        cq_columns = [col for col in df.columns if col.startswith('CQ.') and col != 'CQ.Overall']
        if cq_columns:
            cq_stats = df[cq_columns].describe()
            poor_quality_mask = (df[cq_columns] < 2).any(axis=1)
            poor_quality_pct = (poor_quality_mask.sum() / len(df)) * 100
            print(f"Signal quality analysis:")
            print(f"  - Poor quality samples: {poor_quality_mask.sum()} ({poor_quality_pct:.2f}%)")

            channel_quality = {}
            for col in cq_columns:
                ch_name = col.replace('CQ.', '')
                poor_ch = (df[col] < 2).sum()
                poor_ch_pct = (poor_ch / len(df)) * 100
                channel_quality[ch_name] = (poor_ch_pct, df[col].mean())

            print("  - Channels with poorest quality:")
            worst_channels = sorted(channel_quality.items(), key=lambda x: x[1][0], reverse=True)[:3]
            for ch, (pct, mean_q) in worst_channels:
                print(f"    * {ch}: {pct:.2f}% poor quality, avg quality: {mean_q:.2f}")

        data = df[eeg_channels].values.T

        ch_names = [ch.replace('EEG.', '') for ch in eeg_channels]

        if 'OriginalTimestamp' in df.columns:
            timestamps = df['OriginalTimestamp'].values
            sfreq = 1.0 / np.median(np.diff(timestamps))
        else:
            sfreq = 128.0

        print(f"Sampling frequency: {sfreq:.2f} Hz")

        info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')
        raw = mne.io.RawArray(data, info)

        try:
            montage = mne.channels.make_standard_montage('standard_1020')
            raw.set_montage(montage, match_case=False)
            print("Successfully set 10-20 montage")
        except Exception as e:
            print(f"Warning: Could not set montage: {str(e)}")

        print("\nApplying preprocessing steps:")
        raw_filtered = raw.copy().filter(l_freq=0.5)
        print("  - Applied band-pass filter (1-40 Hz)")

        raw_notch = raw_filtered.copy().notch_filter(freqs=[50, 60])
        print("  - Applied notch filter (50/60 Hz)")

        print("\nSpectral analysis:")
        bands = {
            'delta': (0.5, 4),
            'theta': (4, 8),
            'alpha': (8, 12),
            'beta': (12, 30),
            'gamma': (30, 640) # no max
        }

        band_powers = {}
        for band_name, (fmin, fmax) in bands.items():
            psd = raw_notch.compute_psd(method="welch", fmin=fmin, fmax=fmax,
                                      n_fft=int(sfreq * 2), n_overlap=int(sfreq))
            print("psd:", psd)
            print("psd type:", type(psd))
            #Is psd a number or a time series? we want both

            psds_data = psd.get_data()
            freqs = psd.freqs

            band_powers[band_name] = np.mean(psds_data, axis=1)

        total_power = sum(np.mean(powers) for powers in band_powers.values())
        for band, powers in band_powers.items():
            rel_power = np.mean(powers) / total_power * 100
            print(f"  - {band.capitalize()} power: {rel_power:.2f}%")

        dominant_band = max(band_powers.items(), key=lambda x: np.mean(x[1]))
        print(f"  - Dominant frequency band: {dominant_band[0].capitalize()}")

        print("\nRunning ICA for artifact removal:")
        ica = ICA(n_components=min(15, len(ch_names)), random_state=42)
        ica.fit(raw_notch)

        try:
            eog_indices, eog_scores = ica.find_bads_eog(raw_notch)
            if eog_indices:
                ica.exclude = eog_indices
                print(f"  - Identified {len(eog_indices)} components related to eye artifacts")
        except Exception as e:
            print(f"  - Could not automatically identify eye artifacts: {str(e)}")

        raw_ica = raw_notch.copy()
        ica.apply(raw_ica)
        print("  - Successfully applied ICA correction")

        if identifier in logs and logs[identifier]:
            print(f"\nEvent analysis for recording {identifier}:")

            first_timestamp = df['OriginalTimestamp'].iloc[0] if 'OriginalTimestamp' in df.columns else df.index[0]

            event_types = {}
            for timestamp, color, description in logs[identifier]:
                event_type = None
                if '+' in description:
                    event_type = 'cross'
                elif 'first word' in description.lower() and color == 'r':
                    event_type = 'first_word_light'
                elif 'first word' in description.lower() and color == 'b':
                    event_type = 'first_word_dark'
                elif 'second word' in description.lower() and color == 'r':
                    event_type = 'second_word_light'
                elif 'second word' in description.lower() and color == 'b':
                    event_type = 'second_word_dark'

                if event_type:
                    event_types[event_type] = event_types.get(event_type, 0) + 1

            for event_type, count in event_types.items():
                print(f"  - {event_type}: {count} occurrences")

            words = set()
            for _, _, description in logs[identifier]:
                if "word '" in description.lower():
                    parts = description.split("'")
                    if len(parts) >= 2:
                        word = parts[1].strip()
                        words.add(word)

            if words:
                print(f"  - Unique words presented: {', '.join(words)}")
        else:
            print(f"\nNo event logs found for recording {identifier}")

        print("\nGenerating visualization...")

        data_clean = raw_ica.get_data()
        times = raw_ica.times

        fig = go.Figure()

        channels_to_plot = min(5, len(ch_names))
        channel_indices = np.linspace(0, len(ch_names)-1, channels_to_plot).astype(int)

        for i in channel_indices:
            ch = ch_names[i]
            # Scale and offset each channel for better visualization
            scaled_data = data_clean[i] * 1e6  # Convert to µV
            offset = i * 50  # Offset for visualization

            fig.add_trace(go.Scatter(
                x=times,
                y=scaled_data + offset,
                name=ch,
                line=dict(width=1)
            ))

        if identifier in logs and logs[identifier]:
          for timestamp, color, description in logs[identifier]:
              event_time = timestamp - first_timestamp

              if event_time >= 0 and event_time <= times[-1]:
                  marker_text = ""
                  marker_color = color

                  if '+' in description:
                      marker_text = "+"
                  elif "lightblue" in description.lower():
                      marker_text = "overt"
                  elif "blue" in description.lower():
                      marker_text = "covert"

                  try:
                      fig.add_vline(
                          x=event_time,
                          line=dict(color=marker_color, width=1, dash="dash"),
                          annotation_text=marker_text,
                          annotation_position="top right"
                      )
                  except ValueError as e:
                      print(f"  - Warning: Could not add marker at {event_time}s: {str(e)}")
                      fig.add_vline(
                          x=event_time,
                          line=dict(color="gray", width=1, dash="dash"),
                          annotation_text=marker_text,
                          annotation_position="top right"
                      )

        fig.update_layout(
            title=f"Clean EEG Data with Event Markers - {os.path.basename(file_path)}",
            xaxis_title="Time (s)",
            yaxis_title="Amplitude (µV + offset)",
            height=600,
            showlegend=True
        )

        output_file = f"{results_dir}/{os.path.basename(file_path).replace('.csv', '_processed.html')}"
        fig.write_html(output_file)
        print(f"Interactive visualization saved to: {output_file}")

        print("\nSUMMARY FINDINGS:")
        print(f"  - Recording duration: {times[-1]:.2f} seconds")
        print(f"  - Channels analyzed: {len(ch_names)}")

        # Calculate SNR (signal-to-noise ratio) - simplified approach
        # Using alpha band power during fixation vs. during word presentation as a proxy
        if identifier in logs and logs[identifier]:
            try:
                cross_periods = []
                word_periods = []

                for timestamp, _, description in logs[identifier]:
                    rel_time = timestamp - first_timestamp
                    if rel_time >= 0 and rel_time <= times[-1]:
                        time_idx = int(rel_time * sfreq)
                        if '+' in description:
                            end_idx = min(time_idx + int(sfreq), len(times))
                            cross_periods.append((time_idx, end_idx))
                        elif 'word' in description.lower() and not 'again' in description.lower():
                            end_idx = min(time_idx + int(2 * sfreq), len(times))
                            word_periods.append((time_idx, end_idx))

                if cross_periods and word_periods:
                    alpha_cross = []
                    for start, end in cross_periods:
                        for ch in range(len(ch_names)):
                            # Simple FFT-based power in 8-13 Hz
                            if end - start > 0:  # Make sure window has data
                                segment = data_clean[ch, start:end]
                                fft_vals = np.abs(np.fft.rfft(segment))
                                fft_freq = np.fft.rfftfreq(len(segment), 1.0/sfreq)

                                # Get alpha band power
                                alpha_idx = np.logical_and(fft_freq >= 8, fft_freq <= 13)
                                alpha_power = np.mean(fft_vals[alpha_idx]**2)
                                alpha_cross.append(alpha_power)

                    # Calculate alpha power during word presentation
                    alpha_word = []
                    for start, end in word_periods:
                        for ch in range(len(ch_names)):
                            if end - start > 0:  # Make sure window has data
                                segment = data_clean[ch, start:end]
                                fft_vals = np.abs(np.fft.rfft(segment))
                                fft_freq = np.fft.rfftfreq(len(segment), 1.0/sfreq)

                                # Get alpha band power
                                alpha_idx = np.logical_and(fft_freq >= 8, fft_freq <= 13)
                                alpha_power = np.mean(fft_vals[alpha_idx]**2)
                                alpha_word.append(alpha_power)

                    if alpha_cross and alpha_word:
                        alpha_cross_mean = np.mean(alpha_cross)
                        alpha_word_mean = np.mean(alpha_word)

                        # Calculate alpha ERD (event-related desynchronization)
                        alpha_erd_pct = ((alpha_cross_mean - alpha_word_mean) / alpha_cross_mean) * 100

                        if alpha_erd_pct > 0:
                            print(f"  - Alpha ERD during word presentation: {alpha_erd_pct:.2f}%")
                            if alpha_erd_pct > 15:
                                print("    * Strong alpha suppression suggests attentive processing")
                            elif alpha_erd_pct > 5:
                                print("    * Moderate alpha suppression observed")
                            else:
                                print("    * Minimal alpha suppression observed")
                        else:
                            print(f"  - Alpha ERS during word presentation: {-alpha_erd_pct:.2f}%")
                            print("    * Unusual alpha synchronization - potential artifact or unique task effect")
            except Exception as e:
                print(f"  - Could not calculate alpha ERD/ERS: {str(e)}")

        return {
            'success': True,
            'file': os.path.basename(file_path),
            'channels': len(ch_names),
            'samples': len(times),
            'duration': times[-1],
            'events': len(logs.get(identifier, [])),
            'visualization': output_file
        }

    except Exception as e:
        print(f"ERROR during processing: {str(e)}")
        traceback.print_exc()
        return {
            'success': False,
            'file': os.path.basename(file_path),
            'error': str(e)
        }

In [15]:
name = 'Aws Safi'
print("\nScanning data directory...")
files_to_process = []

for s in os.listdir(data_path):
    if not s.endswith('.csv'):
        continue
    if not s.startswith(name):
        continue

    identifier = s.split(name)[1].split("_")[0]
    file_path = f"{data_path}/{s}"

    is_valid, message = validate_csv(file_path)
    if is_valid:
        files_to_process.append((file_path, identifier))
    else:
        print(f"Skipping invalid file {s}: {message}")

print(f"\nFound {len(files_to_process)} valid files to process")

results = []
for file_path, identifier in files_to_process:
    if identifier in logs:
        result = preprocess_eeg(file_path, identifier, logs)
        if result:
            results.append(result)
    else:
        print(f"Skipping {file_path} - no matching log file found")

successful = [r for r in results if r.get('success', False)]
failed = [r for r in results if not r.get('success', False)]

print(f"\n{'='*80}\nFINAL PROCESSING SUMMARY\n{'='*80}")
print(f"Total files processed: {len(results)}")
print(f"Successfully processed: {len(successful)}")
print(f"Failed processing: {len(failed)}")

if successful:
    print("\nSuccessfully processed files:")
    for i, result in enumerate(successful, 1):
        print(f"{i}. {result['file']} - {result['channels']} channels, {result['duration']:.2f}s, {result['events']} events")

if failed:
    print("\nFailed files:")
    for i, result in enumerate(failed, 1):
        print(f"{i}. {result['file']} - Error: {result['error']}")

print(f"\nAll results saved to: {results_dir}")
print("Complete!")


Scanning data directory...

Found 3 valid files to process

Processing file: Aws Safi1742289648917_EPOCX_211983_2025.03.18T12.20.50+03.00.md.pm.bp.csv
Successfully read data: 86580 rows, 169 columns
ERROR: No EEG channels found in data

Processing file: Aws Safi1742290353356_EPOCX_211983_2025.03.18T12.32.34+03.00.md.pm.bp.csv
Successfully read data: 86609 rows, 169 columns
ERROR: No EEG channels found in data

Processing file: Aws Safi1742291163364_EPOCX_211983_2025.03.18T12.46.04+03.00.md.pm.bp.csv
Successfully read data: 86576 rows, 169 columns
ERROR: No EEG channels found in data

FINAL PROCESSING SUMMARY
Total files processed: 0
Successfully processed: 0
Failed processing: 0

All results saved to: results
Complete!
